In [61]:
import numpy as np
import pandas as pd
from math import isnan, radians, cos, sin, asin, sqrt

import core

In [30]:
class PlaceEntry:
    def __init__(self, name, address, lon, lat):
        self.name = name
        self.address = address
        self.lon = lon
        self.lat = lat
    
    def get_name(self):
        return self.name
    
    def get_address(self):
        return self.address
    
    def get_coords(self):
        return (self.lon, self.lat)
    
    def __repr__(self):
        return f'{self.name}, {self.address}. Coords: ({self.lon}, {self.lat})'

In [31]:
df_food = pd.read_csv('PlacesDatabase/food_places.csv')
df_food.head()

,id,Name_en,IsNetObject_en,OperatingCompany_en,TypeObject_en,AdmArea_en,District_en,Address_en,PublicPhone_en,SeatsCount_en,SocialPrivileges_en,Longitude_WGS84_en,Latitude_WGS84_en,geoData_type
0,0,СМЕТАНА,нет,NaN,NaN,NaN,NaN,"Российская Федерация, город Москва, внутригоро...",[{'PublicPhone_en': ''}],44.0,нет,37.714565,55.879002,Point
1,1,Родник,нет,NaN,NaN,NaN,NaN,"город Москва, улица Талалихина, дом 2/1, корпус 1",[{'PublicPhone_en': ''}],35.0,нет,37.673306,55.738239,Point
2,2,Кафе «Академия»,нет,NaN,NaN,NaN,NaN,"Российская Федерация, город Москва, внутригоро...",[{'PublicPhone_en': ''}],95.0,нет,37.669648,55.735511,Point
3,3,ГБОУ «Школа № 1430 имени Героя Социалистическо...,нет,NaN,NaN,NaN,NaN,"город Москва, Угличская улица, дом 17",[{'PublicPhone_en': ''}],240.0,нет,37.566940,55.904019,Point
4,4,Брусника,да,Брусника,NaN,NaN,NaN,"город Москва, переулок Сивцев Вражек, дом 6/2",[{'PublicPhone_en': ''}],10.0,нет,37.598128,55.747390,Point


In [71]:
places = []

def validate_point(p):
    lat, lon = p
    return (-90 <= lat <= 90) and (-180 <= lon <= 180)

for row in df_food.itertuples():
    lon, lat = row.Longitude_WGS84_en, row.Latitude_WGS84_en
    if validate_point((lon, lat)):
        entry = PlaceEntry(
            row.Name_en, row.Address_en,
            lon, lat
        )
        places.append(entry)

In [72]:
def distance_haversine(p1, p2):
    lat1, lon1 = p1
    lat2, lon2 = p2
    if not validate_point(p1) or not validate_point(p2):
        return None

    R = 6371
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    d = R * c
    return d

In [73]:
def get_nearest(places, coords, N):
    place2dist = {}

    lon, lat = coords
    for place in places:
        p_lon, p_lat = place.get_coords()
        dist = distance_haversine((lon, lat), (p_lon, p_lat))
        place2dist[place] = dist
    
    return sorted(place2dist.items(), key=lambda item: item[1])[:N]

In [74]:
get_nearest(places, (37, 55), 5)

[(Кафе, город Москва, поселение Роговское, деревня Каменка, Центральная улица, дом 25, строение 2. Coords: (36.98527299999999, 55.216999604671024),
  19.341788547890328),
 (Пицца Экспресс, Российская Федерация, город Москва, внутригородская территория поселение Роговское, деревня Каменка, земельный участок 16А. Coords: (36.97441852933707, 55.21653914746253),
  19.442033994131787),
 (Пицца Экспресс, город Москва, поселение Роговское, деревня Каменка, Центральная улица, дом 10А. Coords: (36.99252100000001, 55.219936605482935),
  19.549950310612367),
 (Комбинат дошкольного питания, Российская Федерация, город Москва, внутригородская территория поселение Роговское, посёлок Рогово, Школьная улица, дом 5А. Coords: (37.074666, 55.21289160353575),
  20.639871010056094),
 (Пицца Экспресс, город Москва, поселение Роговское, деревня Кресты, дом 11Б, строение 1. Coords: (37.10438, 55.262748617343554),
  26.046125112711074)]